# Desafio de Análise Exploratória de Dados (EDA) e Pré-processamento
## Dataset: Titanic

* Este notebook contém as etapas de carregamento, renomeação de colunas, solução de desafios de análise estatística e as primeiras ações de limpeza e pré-processamento de dados.

### **Pré-requisito:** Certifique-se de que o arquivo `titanic.csv` foi carregado para o ambiente `/content/` do Colab.

In [ ]:
import pandas as pd
import numpy as np
import os

mapa_colunas = {
    'PassengerId': 'id_passageiro',
    'Survived': 'sobreviveu',
    'Pclass': 'classe_p',
    'Name': 'nome',
    'Sex': 'sexo',
    'Age': 'idade',
    'SibSp': 'irmao_conjuge',
    'Parch': 'pais_filhos',
    'Ticket': 'bilhete',
    'Fare': 'tarifa',
    'Cabin': 'cabine',
    'Embarked': 'embarque'
}

caminho_arquivo = '/content/titanic.csv'

if os.path.exists(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo)
    df = df.rename(columns=mapa_colunas)
    
    print(" DataFrame carregado e colunas renomeadas com sucesso!")
    print("\nNovas colunas:", df.columns.tolist())
    print("\nInício do DataFrame (Head):")
    print(df.head())
else:
    print(f" ERRO: Arquivo 'titanic.csv' não encontrado em {caminho_arquivo} .")
    df = None

✅ DataFrame carregado e colunas renomeadas com sucesso!

Novas colunas: ['id_passageiro', 'sobreviveu', 'classe_p', 'nome', 'sexo', 'idade', 'irmao_conjuge', 'pais_filhos', 'bilhete', 'tarifa', 'cabine', 'embarque']

Início do DataFrame (Head):
   id_passageiro  sobreviveu  classe_p  \
0              1           0         3   
1              2           1         1   
2              3           1         3   
3              4           1         1   
4              5           0         3   

                                                nome    sexo  idade  \
0                            Braund, Mr. Owen Harris    male   22.0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   38.0   
2                             Heikkinen, Miss. Laina  female   26.0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   35.0   
4                           Allen, Mr. William Henry    male   35.0   

   irmao_conjuge  pais_filhos           bilhete   tarifa cabine embarque  


# 🎯 Solução dos Desafios de Análise

Utilizaremos o DataFrame `df` com as colunas em Português para responder às perguntas usando funções de filtragem e agregação do Pandas.

**Desafios:**
1.  Filtrar não sobreviventes da 1ª classe e calcular a porcentagem.
2.  Contar passageiros por `sexo` e `embarque`.
3.  Idade média dos `sobreviventes` e `não sobreviventes`.

In [ ]:
if df is not None:
    total_primeira = df[df['classe_p'] == 1].shape[0]

    nao_sobreviveram_primeira = df[(df['classe_p'] == 1) & (df['sobreviveu'] == 0)].shape[0]

    if total_primeira > 0:
        porcentagem_nao_sobreviveram = (nao_sobreviveram_primeira / total_primeira) * 100
        print("\n--- 1. Análise da 1ª Classe ---")
        print(f"Não Sobreviventes na 1ª Classe: {nao_sobreviveram_primeira}")
        print(f"Total na 1ª Classe: {total_primeira}")
        print(f"Porcentagem de Não Sobreviventes (1ª Classe): **{porcentagem_nao_sobreviveram:.2f}%**")
    else:
        print("Dados da 1ª classe não encontrados.")

    print("\n--- 2. Contagem por Sexo e Porto de Embarque ---")

    contagem_cruzada = pd.crosstab(df['sexo'], df['embarque'], dropna=False)
    
    print("\n=== Contagem Cruzada (Sexo vs. Embarque) ===")
    print(contagem_cruzada)

    print("\n=== Contagem Total por Sexo ===")
    print(df['sexo'].value_counts())

    print("\n--- 3. Idade Média por Status de Sobrevivência ---")

    media_idades = df.groupby('sobreviveu')['idade'].mean()
    
    print(f"Idade Média (Não Sobreviventes = 0): **{media_idades.get(0, 0):.2f} anos**")
    print(f"Idade Média (Sobreviventes = 1): **{media_idades.get(1, 0):.2f} anos**")

else:
    print("Não foi possível executar os desafios. Verifique o carregamento na Célula 2.")


--- 1. Análise da 1ª Classe ---
Não Sobreviventes na 1ª Classe: 80
Total na 1ª Classe: 216
Porcentagem de Não Sobreviventes (1ª Classe): **37.04%**

--- 2. Contagem por Sexo e Porto de Embarque ---

=== Contagem Cruzada (Sexo vs. Embarque) ===
embarque   C   Q    S  NaN
sexo                      
female    73  36  203    2
male      95  41  441    0

=== Contagem Total por Sexo ===
sexo
male      577
female    314
Name: count, dtype: int64

--- 3. Idade Média por Status de Sobrevivência ---
Idade Média (Não Sobreviventes = 0): **30.63 anos**
Idade Média (Sobreviventes = 1): **28.34 anos**


# Limpeza e Pré-processamento (Segunda Parte)

Agora vamos iniciar o pré-processamento, tratando os valores ausentes (NaN) nas colunas críticas:

1.  **`cabine`**: Removida devido à alta proporção de valores faltantes.
2.  **`idade`**: Imputada com a **mediana**, pois é mais robusta contra outliers do que a média.
3.  **`embarque`**: Imputada com o **modo** (valor mais frequente), pois é uma variável categórica.

Ao final, verificamos se todos os valores ausentes foram tratados.

In [ ]:
if df is not None:
    print("--- INICIANDO LIMPEZA ROBUSTA E PRÉ-PROCESSAMENTO ---")

    if 'embarque' in df.columns:
        moda_embarque = df['embarque'].mode()[0] 
        df['embarque'].fillna(moda_embarque, inplace=True)
        print(f" 'embarque' imputado com o Modo ('{moda_embarque}').")
    else:
        print(" Coluna 'embarque' não encontrada para imputação. Pulando...")

    if 'idade' in df.columns:
        mediana_idade = df['idade'].median()
        df['idade'].fillna(mediana_idade, inplace=True)
        print(f" 'idade' imputada com a Mediana ({mediana_idade:.2f}).")
    else:
        print(" Coluna 'idade' não encontrada para imputação. Pulando...")
        
    if 'cabine' in df.columns:
        df.drop('cabine', axis=1, inplace=True)
        print(" Coluna 'cabine' removida.")
    else:
        print(" Coluna 'cabine' já foi removida. Continuando...")
   
    colunas_para_dummies = []
    if 'sexo' in df.columns:
        colunas_para_dummies.append('sexo')
    if 'embarque' in df.columns:
        colunas_para_dummies.append('embarque')
        
    if colunas_para_dummies:
        df_dummies = pd.get_dummies(df[colunas_para_dummies], drop_first=True, prefix=colunas_para_dummies)

        df = pd.concat([df.drop(colunas_para_dummies, axis=1), df_dummies], axis=1)
        print(f" Variáveis categóricas {colunas_para_dummies} codificadas (One-Hot Encoding).")
    else:
        print(" Colunas categóricas já parecem estar codificadas ou foram removidas.")

    print("\n--- Verificação Final de Valores Ausentes ---")
    print(df.isnull().sum())
    
    print("\n Pré-processamento e Codificação concluídos com sucesso!")
    print(f"Dimensões finais: {df.shape}")

--- INICIANDO LIMPEZA ROBUSTA E PRÉ-PROCESSAMENTO ---
 Coluna 'embarque' não encontrada para imputação. Pulando...
 'idade' imputada com a Mediana (28.00).
 Coluna 'cabine' já foi removida. Continuando...
 Colunas categóricas já parecem estar codificadas ou foram removidas.

--- Verificação Final de Valores Ausentes ---
id_passageiro    0
sobreviveu       0
classe_p         0
nome             0
idade            0
irmao_conjuge    0
pais_filhos      0
bilhete          0
tarifa           0
sexo_male        0
embarque_Q       0
embarque_S       0
dtype: int64

 Pré-processamento e Codificação concluídos com sucesso!
Dimensões finais: (891, 12)


C:\Users\Luca\AppData\Local\Temp\ipykernel_3544\1288195365.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['idade'].fillna(mediana_idade, inplace=True)


In [ ]:
if df is not None:
    print("\n--- PASSO 7: CODIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS ---")
    
    colunas_originais = ['sexo', 'embarque']
    
    colunas_para_codificar = [col for col in colunas_originais if col in df.columns]

    if not colunas_para_codificar:
        print(" Variáveis 'sexo' e 'embarque' já foram codificadas ou removidas.")
        print("Continuando a execução com o DataFrame atual.")
        
    else:
        df_dummies = pd.get_dummies(df[colunas_para_codificar], drop_first=True)
        
        df = pd.concat([df.drop(colunas_para_codificar, axis=1), df_dummies], axis=1)

        print(f" Variáveis '{colunas_para_codificar}' codificadas com sucesso.")
        print(f"Novo formato do DataFrame: {df.shape[0]} linhas x {df.shape[1]} colunas.")
        print("\nNovas Colunas (Início):")
        print(df.head(3))


--- PASSO 7: CODIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS ---
 Variáveis 'sexo' e 'embarque' já foram codificadas ou removidas.
Continuando a execução com o DataFrame atual.


In [ ]:
if df is not None:
    print("\n--- PASSO 8: ANÁLISE BIVARIADA (SOBREVIVÊNCIA) ---")

    coluna_sexo_analise = 'sexo_male' 
    
    if coluna_sexo_analise in df.columns:
        sobrevivencia_por_sexo = pd.crosstab(df[coluna_sexo_analise], df['sobreviveu'], normalize='index') * 100
        sobrevivencia_por_sexo.index = ['Mulher (sexo_male=0)', 'Homem (sexo_male=1)'] 
        sobrevivencia_por_sexo.columns = ['Não Sobreviveu (%)', 'Sobreviveu (%)']

        print("\n=== Taxa de Sobrevivência por Sexo (Usando Coluna Codificada) ===")
        print(sobrevivencia_por_sexo.round(2))
    else:
        print(f" A coluna {coluna_sexo_analise} não foi encontrada. Verifique se a Célula 7 (Codificação) foi executada corretamente.")

    sobrevivencia_por_classe = pd.crosstab(df['classe_p'], df['sobreviveu'], normalize='index') * 100
    sobrevivencia_por_classe.columns = ['Não Sobreviveu (%)', 'Sobreviveu (%)']

    print("\n\n=== Taxa de Sobrevivência por Classe (Pclass) ===")
    print(sobrevivencia_por_classe.round(2))
    
    print("\n--- Insights Chave ---")
    print("\nO resultado do `pd.crosstab` confirma que **Mulheres** tinham uma chance de sobrevivência dramaticamente maior (~74%) do que Homens (~19%).") 
    print("Além disso, a **Classe** é um forte preditor, com passageiros da 1ª Classe tendo as maiores taxas de sobrevivência (mais de 62%).")


--- PASSO 8: ANÁLISE BIVARIADA (SOBREVIVÊNCIA) ---

=== Taxa de Sobrevivência por Sexo (Usando Coluna Codificada) ===
                      Não Sobreviveu (%)  Sobreviveu (%)
Mulher (sexo_male=0)               25.80           74.20
Homem (sexo_male=1)                81.11           18.89


=== Taxa de Sobrevivência por Classe (Pclass) ===
          Não Sobreviveu (%)  Sobreviveu (%)
classe_p                                    
1                      37.04           62.96
2                      52.72           47.28
3                      75.76           24.24

--- Insights Chave ---

O resultado do `pd.crosstab` confirma que **Mulheres** tinham uma chance de sobrevivência dramaticamente maior (~74%) do que Homens (~19%).
Além disso, a **Classe** é um forte preditor, com passageiros da 1ª Classe tendo as maiores taxas de sobrevivência (mais de 62%).
